## Final Project Submission

Please fill out:
* Student name: Lucas Wilkerson
* Student pace: Flex (self paced / part time)
* Scheduled project review date/time: 
* Instructor name: Morgan Jones
* Blog post URL:


# Microsoft Movie Analysis 

## Overview
(A one-paragraph overview of the project, including the business problem, data, methods, results and recommendations.)

This project analyzes characteristics and factors of movies that are correlated to success at the box office to generate insights and recommendations for the stakeholder. Descriptive analysis of budget, gross earnings, genre and run time will be explored to see how these are related to movie success and return on investment. Microsoft stakeholders can use this analyis to decide what type of films to create and what budgets to plan for.

## Business Problem
(Summary of the business problem you are trying to solve, and the data questions that you plan to answer to solve them.)

***
Questions to consider:
* What are the business's pain points related to this project?
* How did you pick the data analysis question(s) that you did?
* Why are these questions important from a business perspective?
***


Microsoft is planning to create a new movie studio to get into the original video/film space. They do not have any background in creating movies and what makes a successful film. Through exploring data from sources such as Box Office Mojo, IMDB, Rotten Tomatoes, TheMovieDB, and The Numbers, I plan to explore what types of films are doing the best at the box office and find out what characterics are correlated to movie success. These finding will be translated into actionable insights that can help Microsoft's new movie studio decide what direction to take when creating new films. 

## Data Understanding
(Describe the data being used for this project.)

***
Questions to consider:
* Where did the data come from, and how do they relate to the data analysis questions?
* What do the data represent? Who is in the sample and what variables are included?
* What is the target variable?
* What are the properties of the variables you intend to use?
***

In [ ]:
#Import Standard Packages
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
%matplotlib inline
import sqlite3 
from zipfile import ZipFile 

In [ ]:
#Importing Imdb Dataset

#with ZipFile(r"/Users/lucaswilkerson/Desktop/Phase1_project/dsc-phase-1-project-v2-4/zippedData/im.db.zip", 'r') as zfiles:
#   zfiles.printdir()
#   zfiles.extractall()

#Reading in additional files need from The Numbers Budget Dataset
#tn_budgets = pd.read_csv("zippedData/tn.movie_budgets.csv.gz")

## Data Preparation

Describe and justify the process for preparing the data for analysis.

***
Questions to consider:
* Were there variables you dropped or created?
* How did you address missing values or outliers?
* Why are these choices appropriate given the data and the business problem?

## Evaluation
Evaluate how well your work solves the stated business problem.

***
Questions to consider:
* How do you interpret the results?
* How well does your model fit your data? How much better is this than your baseline model?
* How confident are you that your results would generalize beyond the data you have?
* How confident are you that this model would benefit the business if put into use?
***

## Conclusions
Provide your conclusions about the work you've done, including any limitations or next steps.

***
Questions to consider:
* What would you recommend the business do as a result of this work?
* What are some reasons why your analysis might not fully solve the business problem?
* What else could you do in the future to improve this project?
***